# Lesson 2: Langchain Expression language

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Simple Chain

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!'

# More complex chain

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [4]:
retriever.get_relevant_documents("what do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [5]:
template = """Answer the question based only on the following context:
{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [6]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [7]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

# Bind

In [8]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    }
]

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [10]:
runnable = prompt | model

runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{"airport_code":"SFO"}'}})

In [11]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"]
        }
    }
]

In [12]:
model = model.bind(functions=functions)

runnable = prompt | model
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{"team_name":"patriots"}'}})

# Fallbacks

In [13]:
from langchain.llms import OpenAI
import json

simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)

In [14]:
def ensure_valid_json(response):
    try:
        parsed_response = json.loads(response)
        return parsed_response
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON"}


simple_chain = simple_model | ensure_valid_json

challenge = """write three poems in a JSON blob, where each poem is a JSON blob with keys 'title', 'author', and 'first_line'.
Output the entire response in a valid JSON format.
"""

simple_chain.invoke(challenge)

{'poem1': {'title': 'Autumn Leaves',
  'author': 'Jane Smith',
  'first_line': 'Crisp leaves crunch underfoot'},
 'poem2': {'title': "The Ocean's Song",
  'author': 'John Doe',
  'first_line': 'Waves crash against the shore'},
 'poem3': {'title': "A Winter's Night",
  'author': 'Emily Jones',
  'first_line': 'Snowflakes dance in the moonlight'}}

In [15]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [16]:
chain.invoke(challenge)

{'poems': [{'title': 'The Rose',
   'author': 'Emily Dickinson',
   'first_line': 'A rose is a rose is a rose'},
  {'title': 'The Road Not Taken',
   'author': 'Robert Frost',
   'first_line': 'Two roads diverged in a yellow wood'},
  {'title': 'Hope is the Thing with Feathers',
   'author': 'Emily Dickinson',
   'first_line': 'Hope is the thing with feathers'}]}

In [17]:
final_chain = simple_chain.with_fallbacks([chain])

In [18]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Autumn Leaves',
  'author': 'Jane Smith',
  'first_line': 'Crisp leaves crunch underfoot'},
 'poem2': {'title': "The Ocean's Song",
  'author': 'John Doe',
  'first_line': 'Waves crash against the shore'},
 'poem3': {'title': "A Winter's Night",
  'author': 'Emily Jones',
  'first_line': 'Snowflakes dance in the moonlight'}}

# Interface

In [19]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bears"})

'Why did the bear dissolve in water?\n\nBecause it was polar!'

In [20]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

for t in chain.stream({"topic": "bears"}):
    print(t)



Why
 did
 the
 bear
 break
 up
 with
 his
 girlfriend
?

Because
 she
 was
 too
 unbearable
!



In [21]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend?\nBecause he couldn't bear the distance!"